In [7]:
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.linear_model import LogisticRegression
import numpy as np
import _pickle as pickle

In [2]:
goodreads_irony_f = open("goodreads_irony_edited.pickle", mode="rb")
goodreads_knowledge_f = open("goodreads_knowledge_keep.pickle", mode="rb")
goodreads_metaphor_f = open("goodreads_metaphor_keep.pickle", mode="rb")

goodreads_irony = pickle.load(goodreads_irony_f)
goodreads_knowledge = pickle.load(goodreads_knowledge_f)
goodreads_metaphor = pickle.load(goodreads_metaphor_f)

goodreads_irony_f.close()
goodreads_knowledge_f.close()
goodreads_metaphor_f.close()

ironic = goodreads_irony
ironic_labels = np.ones(len(ironic))
non_ironic = goodreads_knowledge + goodreads_metaphor
non_ironic_labels = np.zeros(len(non_ironic))
full_set = ironic + non_ironic
full_labels = np.concatenate([ironic_labels, non_ironic_labels])

ironic_size = len(ironic)
non_ironic_size = len(non_ironic)
full_set_size = len(full_set)

print("Ironic examples: " + str(ironic_size))
print("Non ironic examples: " + str(non_ironic_size))
print("Full set (for validation): " + str(full_set_size))

Ironic examples: 320
Non ironic examples: 807
Full set (for validation): 1127


In [3]:
full_set_np = np.asarray(full_set)

train_features = np.concatenate([full_set_np[0:240], full_set_np[320:925]])
train_labels = np.concatenate([np.ones(240), np.zeros(605)])

test_features = np.concatenate([full_set_np[240:320], full_set_np[925:1127]])
test_labels = np.concatenate([np.ones(80), np.zeros(202)])

print(len(train_features))
print(len(train_labels))
print(len(test_features))
print(len(test_labels))

845
845
282
282


In [4]:
embed = hub.Module('./embeddings/NNLM50')
test_messages = ["The quick brown fox jumped over the lazy dog"]

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    message_embeddings = session.run(embed(test_messages))
message_embeddings

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0418 19:58:24.383136  6016 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


array([[ 0.41665807, -0.26561025,  0.00393612, -0.01199844,  0.00838736,
        -0.12759466,  0.06728889, -0.09666663,  0.05915619, -0.11202545,
         0.34699073,  0.0915171 , -0.06559876,  0.24825054, -0.15707098,
        -0.29335728,  0.29755536,  0.17687456,  0.06018243, -0.49035624,
        -0.26726544, -0.08323846, -0.10170481,  0.2833382 ,  0.3111493 ,
         0.0923718 , -0.20664847,  0.0575049 ,  0.18080623,  0.02725045,
         0.00090702, -0.11329008,  0.06102838, -0.1938533 , -0.2558603 ,
        -0.12667309, -0.03718634,  0.07071813,  0.14304037, -0.28412327,
         0.18178123, -0.10954207,  0.13003   , -0.15912755, -0.10998374,
        -0.0360543 , -0.17689341,  0.25698033,  0.04685439, -0.15045254]],
      dtype=float32)

In [6]:
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    x_train = session.run(embed(train_features))
    x_test = session.run(embed(test_features))
y_train = train_labels
y_test = test_labels
print("x train shape: " + str(x_train.shape))
print("x test shape: " + str(x_test.shape))
print("y train shape: " + str(y_train.shape))
print("y test shape: " + str(y_test.shape))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0418 20:04:14.054710  6016 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0418 20:04:14.105601  6016 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


x train shape: (845, 50)
x test shape: (282, 50)
y train shape: (845,)
y test shape: (282,)


In [8]:
classifier = LogisticRegression(random_state=0, solver='liblinear', multi_class='ovr').fit(x_train, y_train)

In [9]:
classifier.score(x_train,y_train)

0.8804733727810651

In [10]:
classifier.score(x_test,y_test)

0.8900709219858156